# load

In [ ]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F



class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model = CNNPointDetector()
# 初始化模型
model = CNNPointDetector()
model.load_state_dict(torch.load('fan_corner_detector_1127.pth'))  # 加载模型参数
model.train()

# newstart

In [4]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points1_1127.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model = CNNPointDetector()

In [6]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min left')

Epoch [1/500], Loss: 5197524.8571 about 500.0 min left
Epoch [2/500], Loss: 5166818.0714 about 499.0 min left
Epoch [3/500], Loss: 4994906.2143 about 498.0 min left
Epoch [4/500], Loss: 4418717.0000 about 497.0 min left
Epoch [5/500], Loss: 2992824.8214 about 496.0 min left
Epoch [6/500], Loss: 817614.2433 about 495.0 min left
Epoch [7/500], Loss: 563564.5536 about 494.0 min left
Epoch [8/500], Loss: 252966.1250 about 493.0 min left
Epoch [9/500], Loss: 284405.6339 about 492.0 min left
Epoch [10/500], Loss: 216089.5201 about 491.0 min left
Epoch [11/500], Loss: 195811.6629 about 490.0 min left
Epoch [12/500], Loss: 186297.1027 about 489.0 min left
Epoch [13/500], Loss: 182254.6975 about 488.0 min left
Epoch [14/500], Loss: 177901.3214 about 487.0 min left
Epoch [15/500], Loss: 178852.7254 about 486.0 min left
Epoch [16/500], Loss: 175836.5179 about 485.0 min left
Epoch [17/500], Loss: 175117.4821 about 484.0 min left
Epoch [18/500], Loss: 175322.6562 about 483.0 min left
Epoch [19/500]

In [7]:
torch.save(model.state_dict(), './fan_corner1_detector_1127.pth')

# test

In [8]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points1_1126test.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [9]:
def evaluate(model, dataset, num_samples=30):
    model.eval()
    with torch.no_grad():
        for i in range(num_samples):
            img, label = dataset[i]
            img = img.unsqueeze(0)  # Add batch dimension
            
            # Model prediction
            prediction = model(img)
            print(f"True: {label.numpy()}, Predicted: {prediction.numpy().flatten()}")

# Test on some sample
evaluate(model, dataset)

True: [2374. 2429.], Predicted: [2351.567  2462.7986]
True: [2869. 2437.], Predicted: [2756.762  2443.9192]
True: [2907. 2384.], Predicted: [2804.761  2487.8044]
True: [2924. 2309.], Predicted: [2635.2244 2384.4348]
True: [2662. 1697.], Predicted: [2513.6877 1844.5686]
True: [2543. 1531.], Predicted: [2497.7192 1597.2072]
True: [2411. 1355.], Predicted: [2324.6978 1391.3228]
True: [2469. 1651.], Predicted: [2359.071  1841.3215]
True: [2491. 1765.], Predicted: [2451.9521 1871.1104]
True: [2492. 1862.], Predicted: [2483.89   1963.4971]
True: [2219. 2124.], Predicted: [2289.3025 2243.774 ]
True: [2112. 2129.], Predicted: [2309.5825 2227.8542]
True: [2422. 2634.], Predicted: [2408.1357 2203.2864]
True: [1992. 2117.], Predicted: [2126.0957 2190.184 ]
True: [1164. 1824.], Predicted: [1250.1183 1859.046 ]
True: [1252. 1934.], Predicted: [1307.7375 1919.4523]
True: [1394. 2077.], Predicted: [1429.3511 2004.885 ]
True: [2496. 2736.], Predicted: [2503.3147 2443.5493]
True: [1943. 2556.], Predict

# keep training

In [11]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points1_1127.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [12]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 420
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1,'min left')

Epoch [1/420], Loss: 31932.7447 about 420.0 min left
Epoch [2/420], Loss: 26117.0145 about 419.0 min left
Epoch [3/420], Loss: 22334.2458 about 418.0 min left
Epoch [4/420], Loss: 23351.7698 about 417.0 min left
Epoch [5/420], Loss: 21835.6945 about 416.0 min left
Epoch [6/420], Loss: 20738.8680 about 415.0 min left
Epoch [7/420], Loss: 19644.5559 about 414.0 min left
Epoch [8/420], Loss: 19152.3065 about 413.0 min left
Epoch [9/420], Loss: 18227.2659 about 412.0 min left
Epoch [10/420], Loss: 18903.3043 about 411.0 min left
Epoch [11/420], Loss: 17826.2277 about 410.0 min left
Epoch [12/420], Loss: 16494.9434 about 409.0 min left
Epoch [13/420], Loss: 15586.0099 about 408.0 min left
Epoch [14/420], Loss: 16041.7232 about 407.0 min left
Epoch [15/420], Loss: 15426.5650 about 406.0 min left
Epoch [16/420], Loss: 14034.8772 about 405.0 min left
Epoch [17/420], Loss: 14868.6952 about 404.0 min left
Epoch [18/420], Loss: 15516.4576 about 403.0 min left
Epoch [19/420], Loss: 13704.7412 abou

In [13]:
torch.save(model.state_dict(), './fan_corner1_detector_1128.pth')